In [1]:
%matplotlib inline
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Load dataset
df_train = pd.read_csv("input/df_train.csv", index_col=0)
df_test = pd.read_csv("input/df_test.csv", index_col=0)

#Insurance dataset
copy_train = df_train.copy()
copy_test = df_test.copy()

FileNotFoundError: [Errno 2] No such file or directory: 'input/df_train.csv'

In [ ]:
sns.set(style="darkgrid") # set seaborn style

In [7]:
df_train.isna().sum()

time                       0
Madrid_wind_speed          0
Valencia_wind_deg          0
Bilbao_rain_1h             0
Valencia_wind_speed        0
Seville_humidity           0
Madrid_humidity            0
Bilbao_clouds_all          0
Bilbao_wind_speed          0
Seville_clouds_all         0
Bilbao_wind_deg            0
Barcelona_wind_speed       0
Barcelona_wind_deg         0
Madrid_clouds_all          0
Seville_wind_speed         0
Barcelona_rain_1h          0
Seville_pressure           0
Seville_rain_1h            0
Bilbao_snow_3h             0
Barcelona_pressure         0
Seville_rain_3h            0
Madrid_rain_1h             0
Barcelona_rain_3h          0
Valencia_snow_3h           0
Madrid_weather_id          0
Barcelona_weather_id       0
Bilbao_pressure            0
Seville_weather_id         0
Valencia_pressure       2068
Seville_temp_max           0
Madrid_pressure            0
Valencia_temp_max          0
Valencia_temp              0
Bilbao_weather_id          0
Seville_temp  

In [8]:
print(f'Train data shape: {df_train.shape}  |  Test data shape: {df_test.shape}')

Train data shape: (8763, 48)  |  Test data shape: (2920, 47)


In [9]:
missing_values_train = df_train.Valencia_pressure.isna().sum()
print(f'Train data Missing Values: {missing_values_train}  |   Percentage: {round(( missing_values_train/ df_train.Valencia_pressure.shape[0]) *100, 2)}%')

Train data Missing Values: 2068  |   Percentage: 23.6%


In [10]:
missing_values_test = df_test.Valencia_pressure.isna().sum()
print(f'Train data Missing Values: {missing_values_test}  |   Percentage: {round(( missing_values_test / df_train.Valencia_pressure.shape[0]) *100, 2)}%')

Train data Missing Values: 454  |   Percentage: 5.18%


In [11]:
mode = pd.concat([df_train.Valencia_pressure , df_test.Valencia_pressure]).mode()

In [12]:
#Impute missing values in Valencia_pressure with mean
df_train.Valencia_pressure.fillna(mode[0] , inplace=True)
df_test.Valencia_pressure.fillna(mode[0], inplace=True)

In [13]:
missing_values_train = df_train.Valencia_pressure.isna().sum()
print(f'Train data Missing Values: {missing_values_train}  |   Percentage: {round(( missing_values_train/ df_train.Valencia_pressure.shape[0]) *100, 2)}%')

Train data Missing Values: 0  |   Percentage: 0.0%


In [14]:
print(f'Sum of unique object: {df_train.Valencia_wind_deg.value_counts().count()}')
df_train.Valencia_wind_deg.unique()

Sum of unique object: 10


array(['level_5', 'level_10', 'level_9', 'level_8', 'level_7', 'level_6',
       'level_4', 'level_3', 'level_1', 'level_2'], dtype=object)

In [15]:
print(f'Sum of unique object: {df_train.Seville_pressure.value_counts().count()}')
df_train.Seville_pressure.unique()

Sum of unique object: 25


array(['sp25', 'sp23', 'sp24', 'sp21', 'sp16', 'sp9', 'sp15', 'sp19',
       'sp22', 'sp11', 'sp8', 'sp4', 'sp6', 'sp13', 'sp17', 'sp20',
       'sp18', 'sp14', 'sp12', 'sp5', 'sp10', 'sp7', 'sp3', 'sp2', 'sp1'],
      dtype=object)

In [16]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder , StandardScaler , MinMaxScaler ,RobustScaler
from sklearn.model_selection import train_test_split , cross_validate

In [17]:
#Impute Categirical features using OrdinalEncoder()
enc = OrdinalEncoder()

In [18]:
df_train.Valencia_wind_deg = enc.fit_transform(df_train[['Valencia_wind_deg']])
df_train.Seville_pressure = enc.fit_transform(df_train[['Seville_pressure']])

In [19]:
df_test.Valencia_wind_deg = enc.fit_transform(df_test[['Valencia_wind_deg']])
df_test.Seville_pressure = enc.fit_transform(df_test[['Seville_pressure']])

In [20]:
df_test.dtypes.value_counts()

float64    44
int64       2
object      1
dtype: int64

In [21]:
# Transform Features

import datetime as dt

df_train['time'] = pd.to_datetime(df_train['time'])
df_test['time'] = pd.to_datetime(df_test['time'])
df_test_copy = df_test.copy()


# day
df_train['Day'] = df_train['time'].dt.day
df_test['Day'] = df_test['time'].dt.day

# month
df_train['Month'] = df_train['time'].dt.month
df_test['Month'] = df_test['time'].dt.month

# year
df_train['Year'] = df_train['time'].dt.year
df_test['Year'] = df_test['time'].dt.year

# hour
df_train['Start_hour'] = df_train['time'].dt.hour
df_test['Start_hour'] = df_test['time'].dt.hour

# Drop Feature
df_train.drop(['time'] , axis=1 , inplace=True)
df_test.drop(['time'] , axis=1 , inplace=True)

In [22]:
columns = df_train.drop(['load_shortfall_3h'] , axis=1).columns

In [23]:
df_train.head()

,Madrid_wind_speed,Valencia_wind_deg,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_clouds_all,Bilbao_wind_speed,Seville_clouds_all,Bilbao_wind_deg,...,Barcelona_temp_min,Bilbao_temp_max,Seville_temp_min,Madrid_temp,Madrid_temp_min,load_shortfall_3h,Day,Month,Year,Start_hour
0,0.666667,5.0,0.0,0.666667,74.333333,64.000000,0.0,1.000000,0.0,223.333333,...,281.013000,269.338615,274.254667,265.938000,265.938000,6715.666667,1,1,2015,3
1,0.333333,1.0,0.0,1.666667,78.333333,64.666667,0.0,1.000000,0.0,221.000000,...,280.561667,270.376000,274.945000,266.386667,266.386667,4171.666667,1,1,2015,6
2,1.000000,9.0,0.0,1.000000,71.333333,64.333333,0.0,1.000000,0.0,214.333333,...,281.583667,275.027229,278.792000,272.708667,272.708667,4274.666667,1,1,2015,9
3,1.000000,8.0,0.0,1.000000,65.333333,56.333333,0.0,1.000000,0.0,199.666667,...,283.434104,281.135063,285.394000,281.895219,281.895219,5075.666667,1,1,2015,12
4,1.000000,7.0,0.0,1.000000,59.000000,57.000000,2.0,0.333333,0.0,185.000000,...,284.213167,282.252063,285.513719,280.678437,280.678437,6620.666667,1,1,2015,15


In [117]:
#Scale the dataset
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_train.drop(['load_shortfall_3h'] , axis=1).values)
scaled_features_test = scaler.fit_transform(df_test.values)

In [118]:
df_train_scaled = pd.DataFrame(scaled_features, index=df_train.index, columns=columns)
df_test_scaled = pd.DataFrame(scaled_features_test, index=df_test.index , columns=columns)

In [119]:
#Add load_short_fall_3h as last_columns on training data
df_train_scaled['load_shortfall_3h'] = copy_train.load_shortfall_3h.values

In [120]:
#Perform a test_train_split
X = df_train_scaled.drop(['load_shortfall_3h'], axis=1)
y = df_train_scaled.load_shortfall_3h

In [121]:
X.head()

,Madrid_wind_speed,Valencia_wind_deg,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_clouds_all,Bilbao_wind_speed,Seville_clouds_all,Bilbao_wind_deg,...,Bilbao_temp,Barcelona_temp_min,Bilbao_temp_max,Seville_temp_min,Madrid_temp,Madrid_temp_min,Day,Month,Year,Start_hour
0,-0.950708,0.248717,-0.362123,-0.796169,0.516117,0.270621,-1.335491,-0.501451,-0.565065,0.630823,...,-2.505658,-1.218309,-2.621665,-2.125118,-2.405393,-2.309892,-1.675368,-1.602429,-1.226179,-1.090901
1,-1.130863,-1.157189,-0.362123,-0.381412,0.692953,0.298017,-1.335491,-0.501451,-0.565065,0.607959,...,-2.353511,-1.292271,-2.475661,-2.040702,-2.357388,-2.261154,-1.675368,-1.602429,-1.226179,-0.654451
2,-0.770554,1.654623,-0.362123,-0.657917,0.383491,0.284319,-1.335491,-0.501451,-0.565065,0.542632,...,-1.671342,-1.124792,-1.821036,-1.570279,-1.680968,-1.574407,-1.675368,-1.602429,-1.226179,-0.218001
3,-0.770554,1.303146,-0.362123,-0.657917,0.118238,-0.044439,-1.335491,-0.501451,-0.565065,0.398912,...,-0.775541,-0.821553,-0.961406,-0.762967,-0.698057,-0.576488,-1.675368,-1.602429,-1.226179,0.218449
4,-0.770554,0.951670,-0.362123,-0.657917,-0.161751,-0.017043,-1.274045,-0.894581,-0.565065,0.255192,...,-0.611717,-0.693885,-0.804197,-0.748327,-0.828246,-0.708665,-1.675368,-1.602429,-1.226179,0.654899


## Feature Seletion

In [29]:
#Recursive Feature Selection (RFE) with Tree based and Gradient based Estimators

#Buid the Model
import pickle
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge , Lasso
from sklearn.ensemble import RandomForestRegressor 
from sklearn.tree import DecisionTreeRegressor

from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import r2_score , mean_squared_error
from sklearn import metrics

#OLS summary
import statsmodels.formula.api as sm

In [30]:
#Train Test Split
X = df_train.drop(['load_shortfall_3h'],axis=1)
y = df_train.load_shortfall_3h

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [31]:
X_train.shape

(5871, 50)

### Feature Selection: Feature Importance of Random Forest Regressor

In [ ]:
selct_important = SelectFromModel(RandomForestRegressor(n_estimators=100 , random_state=0 , n_jobs=1))
selct_important.fit(X_train , y_train)
selct_important.get_support()

In [ ]:
features = X_train.columns[(selct_important.get_support())]
len(features)

In [ ]:
features

In [ ]:
np.mean(selct_important.estimator_.feature_importances_)

In [ ]:
pd.Series(selct_important.estimator_.feature_importances_.ravel()).hist()

In [ ]:
selct_important.estimator_.feature_importances_

In [ ]:
X_train_sel = selct_important.transform(X_train)
X_test_sel = selct_important.transform(X_test)

In [35]:
import xgboost as xgb

In [ ]:
selct_best_xg = SelectFromModel(xgb.XGBRegressor(objective ='reg:squarederror', n_estimators = 100, random_state=0 , n_jobs=1))
selct_best_xg.fit(X_train , y_train)
selct_best_xg.get_support()

In [ ]:
b = X_train.columns[(selct_best_xg.get_support())]
len(b)

In [ ]:
b

In [ ]:
X_train_xg = selct_best_xg.transform(X_train)
X_test_xg = selct_best_xg.transform(X_test)

In [ ]:
def _xgboost_train(X_train, X_test, y_train, y_test):
    forest = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators = 100, random_state=0 , n_jobs=1)
    forest.fit(X_train , y_train)
    pred = forest.predict(X_test)
    print(f'Train RMSE: { np.sqrt(metrics.mean_squared_error(y_train[:2892], pred))}')
    print(f'R Score Train: {r2_score(y_train[:2892] , pred)}')
    print(f'Test RMSE: { np.sqrt(metrics.mean_squared_error(y_test, pred))}')
    print(f'R Score Test: {r2_score(y_test , pred)}')

In [ ]:
def _randonForest_train(X_train, X_test, y_train, y_test):
    forest = RandomForestRegressor(n_estimators=100 , random_state=0 , n_jobs=1)
    forest.fit(X_train , y_train)
    pred = forest.predict(X_test)
    print(f'Train RMSE: { np.sqrt(metrics.mean_squared_error(y_train[:2892], pred))}')
    print(f'R Score Train: {r2_score(y_train[:2892] , pred)}')
    print(f'Test RMSE: { np.sqrt(metrics.mean_squared_error(y_test, pred))}')
    print(f'R Score Test: {r2_score(y_test , pred)}')

In [ ]:
%%time
_randomForest_train(X_train_xg , X_test_xg , y_train , y_test )

In [ ]:
%%time
_xgboost_train(X_train_xg , X_test_xg , y_train , y_test )

#### Recursive Feature Selection(RFE)

In [ ]:
from sklearn.feature_selection import RFE, RFECV

In [ ]:
r_selection = RFE(RandomForestRegressor(n_estimators=100 , random_state=0 , n_jobs=1),n_features_to_select=10)

In [ ]:
r_selection.fit(X_train , y_train)

In [ ]:
r_selection.get_support()

In [ ]:
features1 = X_train.columns[(r_selection.get_support())]
len(features)

In [ ]:
features1

In [ ]:
r_selection.estimator_.feature_importances_

In [ ]:
X_train_r = r_selection.transform(X_train)
X_test_r = r_selection.transform(X_test)

In [ ]:
%%time
_randomForest_train(X_train_r , X_test_r , y_train , y_test )

In [ ]:
%%time
# All features
_randomForest_train(X_train , X_test , y_train , y_test )

### Feature Selection: GradientBoostRegressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
xgrad = RFE(xgb.XGBRegressor(objective ='reg:squarederror', n_estimators = 100, random_state=0 , n_jobs=1),n_features_to_select=10)
xgrad.fit(X_train , y_train)

In [ ]:
xgrad.get_support()

In [ ]:
features2 = X_train.columns[(xgrad.get_support())]
len(features)

In [ ]:
features2

In [ ]:
xgrad.estimator_.feature_importances_

In [ ]:
X_train_g = xgrad.transform(X_train)
X_test_g = xgrad.transform(X_test)

In [ ]:
%%time
_randomForest_train(X_train_g , X_test_g , y_train , y_test )

In [ ]:
%%time
#All features
_randomForest_train(X_train , X_test , y_train , y_test )

### Feature Selection: Random Forest Tree & Gardient Boost - 50 Iterations

In [ ]:
print('>'*40 + 'GradientBoostRegressor'+ '<'*40)
for index in range(1 , 51):
    grad_selection = RFE(GradientBoostingRegressor(n_estimators=100 , random_state=0), n_features_to_select=index)
    grad_selection.fit(X_train , y_train)
    X_train_g = grad_selection.transform(X_train)
    X_test_g = grad_selection.transform(X_test)
    print(f'Selected Features: {index}')
    _randomForest_train(X_train_g , X_test_g , y_train , y_test )
    print()

In [ ]:
print('>'*40 + 'RandomForestRegressor'+ '<'*40)
for index in range(1 , 51):
    grad_selection = RFE(RandomForestRegressor(n_estimators=100 , random_state=0), n_features_to_select=index)
    grad_selection.fit(X_train , y_train)
    X_train_r = grad_selection.transform(X_train)
    X_test_r = grad_selection.transform(X_test)
    print(f'Selected Features: {index}')
    _randomForest_train(X_train_r , X_test_r , y_train , y_test )
    print()

In [ ]:
grad_sel = RFECV(GradientBoostingRegressor(n_estimators=100 , random_state=0), cv=5)
grad_sel.fit(X_train , y_train)
X_train_g = grad_sel.transform(X_train)
X_test_g = grad_sel.transform(X_test)
print(f'Selected Features: {4}')
_randomForest_train(X_train_g , X_test_g , y_train , y_test )

In [ ]:
best_feat1 = X_train.columns[(grad_sel.get_support())]
len(best_feat1)

In [ ]:
best_feat1

In [ ]:
forest_selection = RFECV(RandomForestRegressor(n_estimators=100 , random_state=0, n_jobs=1), cv=5)
forest_selection.fit(X_train , y_train)
X_train_g = forest_selection.transform(X_train)
X_test_g = forest_selection.transform(X_test)
print(f'Selected Features: {10}')
_randomForest_train(X_train_g , X_test_g , y_train , y_test )

In [ ]:
best_feat2 = X_train.columns[(forest_selection.get_support())]
len(best_feat2)

In [ ]:
best_feat2

In [122]:
best_features =  [ 'Madrid_wind_speed','Bilbao_rain_1h',
                                        'Bilbao_pressure',  'Year', 'Day',
                                        'Month', 'Start_hour']

In [123]:
X = df_test[best_features]
X.columns

Index(['Madrid_wind_speed', 'Bilbao_rain_1h', 'Bilbao_pressure', 'Year', 'Day',
       'Month', 'Start_hour'],
      dtype='object')

In [126]:
#Train a model
regression = [
    xgb.XGBRegressor(objective ='reg:squarederror', n_estimators = 100, random_state=0 ),
    # LinearRegression(),
    RandomForestRegressor(max_depth=100)
    ]

In [127]:
from sklearn import metrics
print('-'*40 + 'Trained Succesfully' + '-'*40 + '\n')
for items in regression:
    model = items
    model.fit(X_train[best_features],y_train)
    train_model = model.predict(X_train[best_features])
    test_model = model.predict(X_test[best_features])
    
  
    #Dictionary of results
    results_dict = {'Training':
                        {
                            "RMSE": np.sqrt(metrics.mean_squared_error(y_train, train_model)),
                            'R2 Score': r2_score(y_train ,train_model)
                            
                        },
                    'Test':
                        {
                            "RMSE": np.sqrt(metrics.mean_squared_error(y_test, test_model)),
                            'R2 Score': r2_score(y_test ,test_model)

                        }
                    
                   }
    scores = pd.DataFrame(data=results_dict)
    print(scores)
    print('-'*50+'End'+'-'*50)
    print()
    y_pred =  model.predict(X)
    daf = pd.DataFrame(y_pred, columns=['load_shortfall_3h'])
    output = pd.DataFrame({"time": df_test_copy['time'].reset_index(drop=True)})
    submissionF = output.join(daf)
    submissionF['load_shortfall_3h'] = daf.values
    submissionF.to_csv("submissionF.csv", index=False)
    print(submissionF)



----------------------------------------Trained Succesfully----------------------------------------

             Training         Test
RMSE      1540.986567  2800.560417
R2 Score     0.911675     0.718981
--------------------------------------------------End--------------------------------------------------

                    time  load_shortfall_3h
0    2018-01-01 00:00:00       11364.571289
1    2018-01-01 03:00:00        4487.770508
2    2018-01-01 06:00:00        5171.218262
3    2018-01-01 09:00:00        6418.197266
4    2018-01-01 12:00:00       11542.001953
...                  ...                ...
2915 2018-12-31 09:00:00        6120.961426
2916 2018-12-31 12:00:00        9883.640625
2917 2018-12-31 15:00:00        8572.275391
2918 2018-12-31 18:00:00        7231.488770
2919 2018-12-31 21:00:00        9858.897461

[2920 rows x 2 columns]
             Training         Test
RMSE      1079.892780  2947.639697
R2 Score     0.956624     0.688689
-------------------------------